In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import csv
import random
import matplotlib.pyplot as plt
import statistics 
from scipy import stats

plt.rcParams["figure.figsize"] = (6,6)



# THE GRAPH CONTROL PANEL

In [2]:
N =300  # Number of nodes
setting='home'    #the setting we are evaluating(home,work,school,other)
D=100  #number of days the model runs
increase_rate=.05 #the rate of increasing weight per contact
lower_weight=.01  #the minimum weight between two node
upper_weight=3    #the maximum weight between two node
threshold_n=1     #threshold of product weight two nodes with same neighbor for increasing weight
increse_rate_n=0  #increse rate weight for two nodes with same neighbor

## 1-Defining Nodes

In [3]:
G = nx.Graph()

In [4]:
#defining a dictionary of nodes
available_groups  = [] #16_age_groups
group_popularity = [] #based on consensus year 95
v_prevalence=dict() #based on meta analysis(data is not exact!! update needed!!)contan(age_group,prevalance,lower CL)
nodes_dic=dict()

with open ('population.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        available_groups.append(row[0])
        group_popularity.append(float(row[1]))
        
with open ('varicella_prevalance.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        v_prevalence[row[0]]=[float(row[1]),float(row[2])]
        
        


for i in range(N):
    while True:
        r=random.random()
        t=random.randint(0,15)
        if r<=group_popularity[t]:
            nodes_dic[f'n{i+1}']={'age':available_groups[t]}
            break


for i in range(N):
    a=v_prevalence[nodes_dic[f'n{i+1}']['age']][0]
    b=v_prevalence[nodes_dic[f'n{i+1}']['age']][1]
    r=np.random.random()
    t=np.random.normal(a,(a-b)) 
    if r<=t:
        nodes_dic[f'n{i+1}']['infection']=1
    else:
        nodes_dic[f'n{i+1}']['infection']=0
        

In [5]:
#importing data for frequency of contact in each setting for comparison
frequency=dict()
with open ('frequency_contact.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        
        frequency[row[0]]=dict()
        for i in range(1,6):
            frequency[row[0]][i]=float(row[i])

### Which `group` each node belongs to?

* We  used `Age specific population for Iran` to define probability of each group

In [6]:

# let's see the number of nodes in each group:

group_count = {}
for n in nodes_dic.keys():
    
    if nodes_dic[n]['age'] in group_count:
        group_count[nodes_dic[n]['age']] = group_count[nodes_dic[n]['age']] + 1
    else:
        group_count[nodes_dic[n]['age']] = 1
group_count

{'35_39': 29,
 '20_24': 29,
 '40_44': 18,
 '50_54': 17,
 '70_74': 7,
 '30_34': 32,
 '10_14': 28,
 '25_29': 27,
 '5_9': 25,
 '45_49': 12,
 '0_4': 28,
 '60_64': 9,
 '75+': 6,
 '55_59': 7,
 '65_69': 5,
 '15_19': 21}

## 2-Defining Connection Rules between Node Groups

### `We define rules in terms of connection probability between each two groups`


In [7]:
#importing rules from .csv to nested dictionary
rules={}  # rules contain ->{location:{contactor:{contactee:contact number}
temp_0={}
temp_1={}
location=[]
contactor=[]
contactee=[]
contact_number=[]


with open ('contact_rules.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        
        location.append(row[0])
        contactor.append(row[1])
        contactee.append(row[2])
        contact_number.append(float(row[3]))


for o in range (0,1280,256):
    for m in range(o,o+256,16):
        for n in range (m,m+16):
            
            temp_0[contactee[n]]=contact_number[n]
        temp_1[contactor[m]]=temp_0.copy()
    rules[location[o]]=temp_1.copy()     

## 3- Visualization for `SMALL` graph

### ⚠️ Only for SMALL graphs (i.e. noes < 100)

## 4-Defining Rules to Connecting Nodes and Adjusting `Probabilities/Distances`

* This would be the step that our Naive graph would evolve over the time through a simulation process
* we monitor connection between nodes and update weights based on connections

In [8]:
#in this block we've defiend a function "Run" 
def Run(increase_rate,threshold_n,increse_rate_n):
    
    for i in nodes_dic.keys():
        for j in nodes_dic.keys():
            G.add_edge(i,j,weight=1)
            
            
    
    #bulding empty dictinary "contacted" for storing contacts in single day
    #building empty dictionary for monitoring all contacts during all days
    contacted={}
    contacted_all={}
    
    for i in nodes_dic.keys():
        contacted[i]={}
        contacted_all[i]={}
        
        for j in nodes_dic.keys():
            if i!=j:

                contacted[i][j]=0 
                contacted_all[i][j]=[]

            
    
    for d in range (0,D):#we are running the model for 100 days
        #iterating thorough the graph and storing connections in 'contacted'
        
        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                     #age_p is the number of contact between nodes by age
                    age_p=rules[setting][nodes_dic[i]['age']][nodes_dic[j]["age"]]
                
                    r=random.random()
                    if r <(age_p/group_count[nodes_dic[j]['age']])*G[i][j]['weight']:
                        contacted[i][j]=1
                        contacted_all[i][j].append(1)#storing contacts of each run in this dic
                    else:
                        contacted[i][j]=0
                        contacted_all[i][j].append(0)
                        
       # iterating thorough 'contacted' and updating the weights based on it
        
        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                    #decrease_rate=increase_rate*((age_p/(group_count[nodes_dic[j]['age']]-age_p)))
                    if contacted[i][j]==1 and G[i][j]['weight']*(1+increase_rate)<upper_weight:
                        #we don't want the weight to be (?) or more
                        
                        t=G[i][j]['weight']+increase_rate
                        G.add_edge(i,j,weight=t)
                        
        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                    age_p=rules[setting][nodes_dic[i]['age']][nodes_dic[j]["age"]]
                    decrease_rate=increase_rate*age_p/(group_count[nodes_dic[j]['age']]-age_p)
                    if contacted[i][j]==0 and G[i][j]['weight']-decrease_rate >lower_weight : 
                        #we don't want the weight to be 0 or less
                        t=G[i][j]['weight']-decrease_rate
                        G.add_edge(i,j,weight=t)

        #updating weight between two nodes with same neighbor
        #for i in nodes_dic.keys():
            #for j in nodes_dic.keys():
                #if i!=j:
                    #for o in nodes_dic.keys():
                        #if o!=j and o!=i:

                            #if (G[i][o]['weight']*G[i][j]['weight'])>threshold_n and G[o][j]['weight']<1:
                                #t=G[o][j]['weight']+increse_rate_n
                                #G.add_edge(i,j,weight=t)
    return(contacted_all,G)


### Visualize the graph after the updates

# 5- Monitroing the graph
### we monitor the graph for age specific number of contact per day and frequeny of contact

In [9]:
    #this functions returns mean absoloute error 'ages specified number of conatacts'

def metric_A_wilx(contacted_all):
        L1=[]
        L2=[]
    
        rules_m=dict()
        rules_m[setting]=dict()

        for i in rules[setting].keys():
            rules_m[setting][i]=dict()
            for j in rules[setting][i].keys():
                 rules_m[setting][i][j]=0


        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                    t=statistics.mean(contacted_all[i][j])/(group_count[nodes_dic[i]['age']])
                    rules_m[setting][nodes_dic[i]['age']][nodes_dic[j]['age']]+=t
        
        
        #for i in rules_m[setting].keys():
            #for j in rules_m[setting][i].keys():
                
                #print(i,j,'model:',rules_m[setting][i][j],'original:',rules[setting][i][j])

        for i in rules_m[setting].keys():
            for j in rules_m[setting][i].keys():
                L1.append(rules[setting][i][j])
                L2.append(rules_m[setting][i][j])
                
        I=stats.wilcoxon(L1,L2, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
        return(I,rules_m)

In [10]:
def metric_A_abs(contacted_all):
        L=[]
        
    
        rules_m=dict()
        rules_m[setting]=dict()

        for i in rules[setting].keys():
            rules_m[setting][i]=dict()
            for j in rules[setting][i].keys():
                 rules_m[setting][i][j]=0


        for i in nodes_dic.keys():
            for j in nodes_dic.keys():
                if i!=j:
                    t=statistics.mean(contacted_all[i][j])/(group_count[nodes_dic[i]['age']])
                    rules_m[setting][nodes_dic[i]['age']][nodes_dic[j]['age']]+=t
        
        
        #for i in rules_m[setting].keys():
            #for j in rules_m[setting][i].keys():
                
                #print(i,j,'model:',rules_m[setting][i][j],'original:',rules[setting][i][j])

        for i in rules_m[setting].keys():
            for j in rules_m[setting][i].keys():

                
                    L.append(abs((rules_m[setting][i][j])-(rules[setting][i][j])))
                
                
        I=statistics.mean(L)
        return(I,rules_m)

In [11]:
#defining a function that returns mean absoloute error 'frequency of contacts ''
def metric_F(contacted_all):
    n=0
    L1=[]
    L2=[]
    frequency_m=dict()
    for i in frequency.keys():
        frequency_m[i]={1:0,2:0,3:0,4:0,5:0}

    for i in nodes_dic.keys():
        for j in nodes_dic.keys():
            if i!=j:
                t=sum(contacted_all[i][j])/D

                if t==1/D:
                    frequency_m[setting][5]+=1
                    n+=1
                elif t>1/D and t<(1/30) :
                    frequency_m[setting][4]+=1
                    n+=1
                elif t>=(1/30) and t<=(2/30):
                    frequency_m[setting][3]+=1
                    n+=1
                elif t>(2/30) and t <(2/7):
                    frequency_m[setting][2]+=1
                    n+=1
                elif t>=(2/7):
                    n+=1
                    frequency_m[setting][1]+=1

    for i in frequency_m[setting].keys():
        if n!=0:
        
            frequency_m[setting][i]=frequency_m[setting][i]/n
        if n==0:
            
            frequency_m[setting][i]=0
        
            
        
    
    for i in frequency_m[setting].keys():
        
        L1.append(frequency[setting][i])
        L2.append(frequency_m[setting][i])
        
    
    I=stats.wilcoxon(L1,L2, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
    #print('for the first time:',frequency_m[setting][5],'\nless than once a month:',frequency_m[setting][4],'\nabout once or twice a month:',frequency_m[setting][3],'\nabout once or twice a week',frequency_m[setting][2],'\ndaily or almost daily',frequency_m[setting][1])

    return(I,frequency_m)
        


   



# grid-search
### this part of code checks different value sets for function Run 

###### Run(increase_rate,threshold_n,increse_rate_n )

In [12]:
#this part returns the absoloute error for age stratified number of contact
results=Run(increase_rate,threshold_n,increse_rate_n )
M_abs=metric_A_abs(results[0])
print(M_abs[0])

0.009651023198476572


In [13]:
#this part shows number of age stratified contact from data and model for comparision
for i in rules[setting].keys():
    for j in rules[setting].keys():
        print(i,'->',j,M_abs[1][setting][i][j],'//',rules[setting][i][j])

0_4 -> 0_4 0.42999999999999927 // 0.43913
0_4 -> 10_14 0.46321428571428414 // 0.45382
0_4 -> 15_19 0.25392857142857267 // 0.25108
0_4 -> 20_24 0.4274999999999987 // 0.41814
0_4 -> 25_29 0.5792857142857126 // 0.58958
0_4 -> 30_34 0.5492857142857123 // 0.56166
0_4 -> 35_39 0.4374999999999985 // 0.44455
0_4 -> 40_44 0.24500000000000094 // 0.24272
0_4 -> 45_49 0.11607142857142864 // 0.12269
0_4 -> 5_9 0.7232142857142825 // 0.70013
0_4 -> 50_54 0.13464285714285737 // 0.1481
0_4 -> 55_59 0.1335714285714286 // 0.13615
0_4 -> 60_64 0.08500000000000006 // 0.08664
0_4 -> 65_69 0.041428571428571426 // 0.04243
0_4 -> 70_74 0.013928571428571441 // 0.01513
0_4 -> 75+ 0.013571428571428583 // 0.01275
10_14 -> 0_4 0.2932142857142858 // 0.29022
10_14 -> 10_14 0.7735714285714267 // 0.79402
10_14 -> 15_19 0.40857142857142803 // 0.4293
10_14 -> 20_24 0.13071428571428603 // 0.12883
10_14 -> 25_29 0.115357142857143 // 0.11601
10_14 -> 30_34 0.22928571428571556 // 0.23366
10_14 -> 35_39 0.3521428571428566 // 

In [14]:
#this part returns p_value wilcoxon for comparison age stratified number of contacts 

print('%f' % metric_A_wilx(results[0])[0][1])

0.466262


In [15]:
#this shows the mean of all edeges across the graph(it's supposed to stay close to one)
mean_weights=[]
for i in nodes_dic.keys():
    for j in nodes_dic.keys():
        if i!=j:
            
            mean_weights.append(results[1].edges[i,j]['weight'])
            
print(statistics.mean(mean_weights))


1.0003081812235193


In [16]:
#this shows the fitting for frequency
results_F=metric_F(results[0])[1]
for i in results_F[setting].keys():
    print (setting,':',i,'//',results_F[setting][i],'//',frequency[setting][i])

home : 1 // 0.0 // 0.7030300218
home : 2 // 0.014873109912459104 // 0.154702798
home : 3 // 0.1270138827482536 // 0.09122546518
home : 4 // 0.4110000884251481 // 0.03693564104
home : 5 // 0.4471129189141392 // 0.01410607396
